In [1]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.pipeline.wrapper import Module, dsl

### Prepare your workspace

In [2]:
tenant_id = "72f988bf-86f1-41af-91ab-2d7cd011db47"
InteractiveLoginAuthentication(tenant_id=tenant_id)
workspace = Workspace.from_config('config.json')
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id,
      workspace.compute_targets.keys(), sep='\n')

fundamental3
fundamental
eastasia
4f455bd0-f95a-4b7d-8d08-078611508e0b
dict_keys(['myaks2', 'aml-compute', 'my-compute', 'compute-deploy'])


### Choose a compute target

In [3]:
aml_compute_name = 'aml-compute'
try:
    aml_compute = AmlCompute(workspace, aml_compute_name)
    print("Found existing compute target: {}".format(aml_compute_name))
except:
    print("Creating new compute target: {}".format(aml_compute_name))

    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=1, max_nodes=4)
    aml_compute = ComputeTarget.create(workspace, aml_compute_name, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target: aml-compute


### Load data

In [4]:
dataset_name = "THUCNews"
dataset = workspace.datasets[dataset_name]

### Load module

In [5]:
split_data_txt_module_func = Module.from_yaml(workspace, 'split_data_txt/split_data_txt.spec.yaml')
fasttext_train_module_func = Module.from_yaml(workspace, 'fasttext_train/fasttext_train.spec.yaml')
fasttext_evaluation_module_func = Module.from_yaml(workspace, 'fasttext_evaluation/fasttext_evaluation.spec.yaml')
compare_two_models_module_func = Module.from_yaml(workspace, 'compare_two_models/compare_two_models.spec.yaml')

### Construct the pipeline

In [6]:
@dsl.pipeline(name='training_pipeline', description='A sub pipeline including data processing/train/evaluation',
              default_compute_target=aml_compute_name)
def training_pipeline(epochs):
    split_data_txt = split_data_txt_module_func(
        input_dir=dataset,
        training_data_ratio=0.1,
        validation_data_ratio=0.2
    )
    fasttext_train = fasttext_train_module_func(
        training_data_dir=split_data_txt.outputs.training_data_output,
        validation_data_dir=split_data_txt.outputs.validation_data_output,
        epochs=epochs
    )

    fasttext_evaluation = fasttext_evaluation_module_func(
        trained_model_dir=fasttext_train.outputs.trained_model_dir,
        test_data_dir=split_data_txt.outputs.test_data_output
    )

    return {**fasttext_evaluation.outputs, **fasttext_train.outputs}

In [7]:
@dsl.pipeline(name='dummy_automl_pipeline',
              description='A dummy pipeline that trains two models and output the better one',
              default_compute_target=aml_compute_name)
def dummy_automl_pipeline():
    train_and_evalute_model1 = training_pipeline(epochs=5)
    train_and_evalute_model2 = training_pipeline(epochs=10)
    compare = compare_two_models_module_func(
        first_trained_model=train_and_evalute_model1.outputs.trained_model_dir,
        first_trained_result=train_and_evalute_model1.outputs.model_testing_result,
        second_trained_model=train_and_evalute_model2.outputs.trained_model_dir,
        second_trained_result=train_and_evalute_model2.outputs.model_testing_result
    )
    return {**compare.outputs}


In [8]:
# pipeline
pipeline = dummy_automl_pipeline()
# pipeline.save(experiment_name=experiment_name)

In [9]:
# validate
pipeline.validate()

<IPython.core.display.Javascript object>

ValidateView(container_id='container_id_4ca7d72c-2136-404c-a8d7-efa778c01db6_widget', env_json='{"subscription…

{'result': 'validation passed', 'errors': []}

In [10]:
# pipeline_run
experiment_name = 'fasttext_training_process'
pipeline_run = pipeline.submit(experiment_name=experiment_name, regenerate_outputs=False)
# pipeline_run.wait_for_completion()

Submitted PipelineRun 0338930f-7136-484b-b08c-0b30fa40476a
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fasttext_training_process/runs/0338930f-7136-484b-b08c-0b30fa40476a?wsid=/subscriptions/4f455bd0-f95a-4b7d-8d08-078611508e0b/resourcegroups/fundamental/workspaces/fundamental3
